In [1]:
import twitter
from urllib.parse import urlparse
from pprint import pprint as pp

In [2]:
class TwitterAPI(object):
    """
    TwitterAPI class allows the Connection to Twitter via OAuth once you have registered with Twitter and receive the
necessary credentiials
    """
    
    # initialize and get the twitter credentials
    def __init__(self):
        consumer_key = "7i5285rfnOjvf4Aq3himGoajO"
        consumer_secret = "TwFS5yxNcTlCzQ8F0y0pMVeCAKcXzGbY64tMASf8eNz3nNEQLc"
        access_token = "791918143169888256-NqMgohyZTxh7iUSVEOs5zyNC8OLi8qI"
        access_secret = "cHzBhcBMtCN9O494gocInJvkhZTa1l9dOfyc4Xj7vzlQS"
        
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        self.access_token = access_token
        self.access_secret = access_secret
    
        # authenticate credentials with Twitter using OAuth
        self.auth = twitter.oauth.OAuth(access_token, 
                                        access_secret,
                                        consumer_key, 
                                        consumer_secret)
        # creates registered Twitter API
        self.api = twitter.Twitter(auth=self.auth)
    # search Twitter with query q (i.e. "ApacheSpark") and max. result
    def searchTwitter(self, q, max_res=10, **kwargs):
        search_results = self.api.search.tweets(q=q, count=10, **kwargs)
        statuses = search_results['statuses']
        max_results = min(1000, max_res)
        
        for _ in range(10):
            try:
                next_results = search_results['search_metadata']['next_results']
            except KeyError as e:
                break
            
            next_results = urlparse.parse_qsl(next_results[1:])
            kwargs = dict(next_results)
            search_results = self.api.search.tweets(**kwargs)
            statuses += search_results['statuses']
            
            if len(statuses) > max_results:
                break
        return statuses
    # parse tweets as it is collected to extract id, creation, date, user id, tweet text
    def parseTweets(self, statuses):
        return [(status['id'], 
                 status['created_at'], 
                 status['user']['id'],
                 status['text'],
                 url['expanded_url'])
                    for status in statuses
                        for url in status['entities']['urls']
                ]

In [3]:
t = TwitterAPI()

In [4]:
q = "ApacheSpark"
tsearch = t.searchTwitter(q)

URLError: <urlopen error [Errno 101] Network is unreachable>